# Setup

In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import scipy
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import f
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn import metrics
from datetime import datetime
from sklearn.linear_model import Lasso, Ridge, LinearRegression
import pickle

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
plt.style.use('seaborn')

# Clean/Impute/Drop

In [55]:
df.columns

Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'],
      dtype='object')

In [56]:
df = pd.read_csv('life.csv')

# Dropping and Imputing
df2 = df.drop(columns=['Hepatitis B', 'Country', 'Year'])

## Imputing

df2.Population = df.Population.fillna(df.Population.median())
df2.GDP = df.GDP.fillna(df.GDP.median())
df2.Schooling = df.Schooling.fillna(df.Schooling.median())
df2['Total expenditure'] = df['Total expenditure'].fillna(df['Total expenditure'].median())
df2['Income composition of resources'] = df['Income composition of resources'].fillna(df['Income composition of resources'].median())
df2.Alcohol = df.Alcohol.fillna(df.Alcohol.median())

## Dropping

df2 = df2.dropna(subset =['Life expectancy ',])
df2 = df2.dropna()
df2.isna().sum()
corr = df2.corr()
renamed = []
for i in df2.columns:
    renamed.append(i.lower().strip().replace(' ', '_').replace('-','_').replace('/','_'))
rename_dict = dict(zip(df2.columns, renamed))
df2.rename(columns=rename_dict, inplace=True)
shorten = {'life_expectancy':'lifex', \
           'percentage_expenditure':'perc_expend', \
           'total_expenditure':'tot_expend', \
          'population':'pop','income_composition_of_resources':'income_comp'}
df2.rename(columns=shorten, inplace=True)

In [66]:
df2['status'] = np.where(df2['status'] == 'Developing', 0,1)

In [67]:
df2

,status,lifex,adult_mortality,infant_deaths,alcohol,perc_expend,measles,bmi,under_five_deaths,polio,tot_expend,diphtheria,hiv_aids,gdp,pop,thinness__1_19_years,thinness_5_9_years,income_comp,schooling
0,0,65.0,263.0,62,0.01,71.279624,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,0,59.9,271.0,64,0.01,73.523582,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,0,59.9,268.0,66,0.01,73.219243,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,0,59.5,272.0,69,0.01,78.184215,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,0,59.2,275.0,71,0.01,7.097109,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,0,44.3,723.0,27,4.36,0.000000,31,27.1,42,67.0,7.13,65.0,33.6,454.366654,12777511.0,9.4,9.4,0.407,9.2
2934,0,44.5,715.0,26,4.06,0.000000,998,26.7,41,7.0,6.52,68.0,36.7,453.351155,12633897.0,9.8,9.9,0.418,9.5
2935,0,44.8,73.0,25,4.43,0.000000,304,26.3,40,73.0,6.53,71.0,39.8,57.348340,125525.0,1.2,1.3,0.427,10.0
2936,0,45.3,686.0,25,1.72,0.000000,529,25.9,39,76.0,6.16,75.0,42.1,548.587312,12366165.0,1.6,1.7,0.427,9.8


# Model with Dummies (Best Model)

In [68]:
features = df2.drop(columns=['lifex'])

In [72]:
features

,status,adult_mortality,infant_deaths,alcohol,perc_expend,measles,bmi,under_five_deaths,polio,tot_expend,diphtheria,hiv_aids,gdp,pop,thinness__1_19_years,thinness_5_9_years,income_comp,schooling
0,0,263.0,62,0.01,71.279624,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,0,271.0,64,0.01,73.523582,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,0,268.0,66,0.01,73.219243,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,0,272.0,69,0.01,78.184215,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,0,275.0,71,0.01,7.097109,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,0,723.0,27,4.36,0.000000,31,27.1,42,67.0,7.13,65.0,33.6,454.366654,12777511.0,9.4,9.4,0.407,9.2
2934,0,715.0,26,4.06,0.000000,998,26.7,41,7.0,6.52,68.0,36.7,453.351155,12633897.0,9.8,9.9,0.418,9.5
2935,0,73.0,25,4.43,0.000000,304,26.3,40,73.0,6.53,71.0,39.8,57.348340,125525.0,1.2,1.3,0.427,10.0
2936,0,686.0,25,1.72,0.000000,529,25.9,39,76.0,6.16,75.0,42.1,548.587312,12366165.0,1.6,1.7,0.427,9.8


In [69]:
target = (df2.lifex)

In [11]:
renamed = []
for i in features.columns:
    renamed.append(i.lower().strip().replace(' ', '_').replace('-','_').replace('/','_').replace('(','_').replace(')','_').replace("'",'_'))

In [12]:
rename_dict = dict(zip(features.columns, renamed))

In [13]:
rename_dict

{'year': 'year',
 'status': 'status',
 'adult_mortality': 'adult_mortality',
 'infant_deaths': 'infant_deaths',
 'alcohol': 'alcohol',
 'perc_expend': 'perc_expend',
 'measles': 'measles',
 'bmi': 'bmi',
 'under_five_deaths': 'under_five_deaths',
 'polio': 'polio',
 'tot_expend': 'tot_expend',
 'diphtheria': 'diphtheria',
 'hiv_aids': 'hiv_aids',
 'gdp': 'gdp',
 'pop': 'pop',
 'thinness__1_19_years': 'thinness__1_19_years',
 'thinness_5_9_years': 'thinness_5_9_years',
 'income_comp': 'income_comp',
 'schooling': 'schooling',
 'Afghanistan': 'afghanistan',
 'Albania': 'albania',
 'Algeria': 'algeria',
 'Antigua and Barbuda': 'antigua_and_barbuda',
 'Argentina': 'argentina',
 'Armenia': 'armenia',
 'Australia': 'australia',
 'Austria': 'austria',
 'Azerbaijan': 'azerbaijan',
 'Bahamas': 'bahamas',
 'Bahrain': 'bahrain',
 'Bangladesh': 'bangladesh',
 'Barbados': 'barbados',
 'Belarus': 'belarus',
 'Belgium': 'belgium',
 'Belize': 'belize',
 'Benin': 'benin',
 'Bhutan': 'bhutan',
 'Bolivia

In [73]:
features

,status,adult_mortality,infant_deaths,alcohol,perc_expend,measles,bmi,under_five_deaths,polio,tot_expend,diphtheria,hiv_aids,gdp,pop,thinness__1_19_years,thinness_5_9_years,income_comp,schooling
0,0,263.0,62,0.01,71.279624,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,0,271.0,64,0.01,73.523582,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,0,268.0,66,0.01,73.219243,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,0,272.0,69,0.01,78.184215,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,0,275.0,71,0.01,7.097109,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,0,723.0,27,4.36,0.000000,31,27.1,42,67.0,7.13,65.0,33.6,454.366654,12777511.0,9.4,9.4,0.407,9.2
2934,0,715.0,26,4.06,0.000000,998,26.7,41,7.0,6.52,68.0,36.7,453.351155,12633897.0,9.8,9.9,0.418,9.5
2935,0,73.0,25,4.43,0.000000,304,26.3,40,73.0,6.53,71.0,39.8,57.348340,125525.0,1.2,1.3,0.427,10.0
2936,0,686.0,25,1.72,0.000000,529,25.9,39,76.0,6.16,75.0,42.1,548.587312,12366165.0,1.6,1.7,0.427,9.8


In [74]:
features_with_life = features.copy()

In [75]:
features_with_life['lifex'] = df3.lifex

In [76]:
linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select=18)
selector = selector.fit(features, target)

In [77]:
i = selector.ranking_
zipped_rankings = list(zip(i, features))
best_predictors_dum = []
for i in zipped_rankings:
    if i[0] == 1:
        best_predictors_dum.append(i[1])

In [78]:
formula = '+'.join(best_predictors_dum)

In [79]:
best_predictors_dum

['status',
 'adult_mortality',
 'infant_deaths',
 'alcohol',
 'perc_expend',
 'measles',
 'bmi',
 'under_five_deaths',
 'polio',
 'tot_expend',
 'diphtheria',
 'hiv_aids',
 'gdp',
 'pop',
 'thinness__1_19_years',
 'thinness_5_9_years',
 'income_comp',
 'schooling']

#### Train test on model dummies

In [80]:
formula

'status+adult_mortality+infant_deaths+alcohol+perc_expend+measles+bmi+under_five_deaths+polio+tot_expend+diphtheria+hiv_aids+gdp+pop+thinness__1_19_years+thinness_5_9_years+income_comp+schooling'

In [81]:
model = ols(formula = f'lifex~{formula}', data=features_with_life).fit()

In [82]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lifex   R-squared:                       0.820
Model:                            OLS   Adj. R-squared:                  0.819
Method:                 Least Squares   F-statistic:                     725.1
Date:                Mon, 24 Feb 2020   Prob (F-statistic):               0.00
Time:                        15:14:09   Log-Likelihood:                -8123.2
No. Observations:                2888   AIC:                         1.628e+04
Df Residuals:                    2869   BIC:                         1.640e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               53.8167      0.567     94.882      0.000      52.705      54.929
status                   1.4759      0.270      5.476      0.000       0.947       2.004
adult_mortality         -0.0199      0.001    -24.930      0.000      -0.021      -0.018
infant_deaths            0.1004      0.008     11.936      0.000       0.084       0.117
alcohol                  0.0633      0.026      2.435      0.015       0.012       0.114
perc_expend           6.509e-05   9.01e-05      0.722      0.470      -0.000       0.000
measles                -2.1e-05   7.64e-06     -2.750      0.006    -3.6e-05   -6.03e-06
bmi                      0.0410      0.005      8.185      0.000       0.031       0.051
under_five_deaths       -0.0750      0.006    -12.150      0.000      -0.087      -0.063
polio                    0.0265      0.004      5.970      0.000       0.018       0.035
tot_expend               0.0460      0.035      1.328      0.184      -0.022       0.114
diphtheria               0.0344      0.004      7.770      0.000       0.026       0.043
hiv_aids                -0.4654      0.018    -26.507      0.000      -0.500      -0.431
gdp                   3.711e-05   1.37e-05      2.701      0.007    1.02e-05    6.41e-05
pop                   4.811e-10   1.68e-09      0.286      0.775   -2.82e-09    3.78e-09
thinness__1_19_years    -0.0787      0.050     -1.565      0.118      -0.177       0.020
thinness_5_9_years       0.0077      0.050      0.155      0.877      -0.090       0.105
income_comp              5.5241      0.638      8.655      0.000       4.273       6.776
schooling                0.7303      0.045     16.395      0.000       0.643       0.818
==============================================================================
Omnibus:                      144.861   Durbin-Watson:                   0.676
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              399.741
Skew:                          -0.238   Prob(JB):                     1.58e-87
Kurtosis:                       4.760   Cond. No.                     4.84e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.84e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [83]:
features_dum = features[best_predictors_dum]

In [84]:
X_train, X_test, y_train, y_test = train_test_split(features_dum, target, random_state=2, test_size=0.2)

In [85]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [86]:
def run_model(model, X_train, X_test, y_train, y_test):
    price_std = target.std()
    print('Training R^2 :', model.score(X_train, y_train))
    y_pred_train = model.predict(X_train)
    train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_pred_train))
    print('Training Root Mean Square Error', train_rmse)
    print('Training Root Mean Square Error Standardized', train_rmse/price_std)
    print('\n----------------\n')
    print('Testing R^2 :', model.score(X_test, y_test))
    y_pred_test = model.predict(X_test)
    test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred_test))
    print('Testing Root Mean Square Error', test_rmse)
    print('Training Root Mean Square Error Standardized', test_rmse/price_std)

#### Model Results

In [87]:
run_model(model, X_train, X_test, y_train, y_test)

Training R^2 : 0.8186024831221436
Training Root Mean Square Error 4.022108671410732
Training Root Mean Square Error Standardized 0.4235831324550994

----------------

Testing R^2 : 0.8222230496576213
Testing Root Mean Square Error 4.085026274479342
Training Root Mean Square Error Standardized 0.43020921781768634


In [32]:
features_with_life.lifex.describe()

count    2888.000000
mean       69.349377
std         9.495441
min        36.300000
25%        63.475000
50%        72.200000
75%        75.800000
max        89.000000
Name: lifex, dtype: float64